In [1]:
import pandas as pd
import numpy as np
import json
import os
import plotly.express as px
import plotly.graph_objects as go
import time
import datetime

In [2]:
data=pd.read_csv("FINAL_REVWS_ALL.csv",index_col='Unnamed: 0')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cities=['NY','Rome','Stockholm','Los','saint']
markers=["circle","square","diamond","cross","x"]
data=data[data.city.isin(cities)].reset_index(drop=True)
data.date=[time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple()) for s in data.date]

In [4]:
#data=data
data=data[data.date>= 1584738660]
data=data[data.date<= 1596316260]

In [5]:
def to_season(date):
    if date>=1593637860:
        return "июль"
    elif date>=1591045860:
        return "июнь"
    elif date>1588367460:
        return "май"
    elif date>=1585775460:
        return "апрель"
    elif date>=1584738660:
        return "март"

In [6]:
data["месяц"]=[to_season(date) for date in data.date]

In [7]:
df_all=pd.DataFrame()
for city in cities:
    df=data[data.city==city]
    df_mean=df.groupby("месяц").mean().reset_index()[["месяц","rating"]].rename(columns={"rating":"среднее"})
    df_var=df.groupby("месяц").var().reset_index()[["месяц","rating"]].rename(columns={"rating":"дисперсия"})
    df_for_city=df_mean.merge(df_var)
    df_for_city["город"]=[city for i in df_for_city.index]
    myorder = ["март","апрель","май","июнь","июль"]
    df_for_city.index=df_for_city["месяц"]
    df_for_city=df_for_city.reindex(myorder).reset_index(drop=True)
    df_all=pd.concat([df_all,df_for_city],ignore_index=True)

In [8]:
cities_name=pd.DataFrame({
    "город":cities,
    "city":["Нью-Йорк","Рим","Стокгольм","Лос-Анджелес","Санкт-Петербург"]
})
df_all=df_all.merge(cities_name).drop(["город"], axis=1).rename(columns={"city":"город"})

In [20]:
def for_city(fig,all_data,val,title):
    color=0
    for i in range(5):
        data_one=all_data[all_data["город"]==all_data["город"].unique()[i]]
        #print(data_one)
        fig.add_trace(go.Scatter(x=data_one["месяц"], y=data_one[val],
                             marker_symbol=markers[int(color*0.5)], marker_size=10,
                    mode='lines+markers', line=dict(color=px.colors.sequential.Plasma[color]),
                    name=all_data["город"].unique()[i]))
        color+=1
        #fig.update_layout(title=str(data_one.city.unique()[0]),xaxis_title='Алгоритм',
        #           yaxis_title='ndcg10 в городе '+ str(data_one.city.unique()[0]))
    #fig.update_layout(xaxis_title='месяц',
    #            yaxis_title=title)
    fig.update_xaxes(title_text="месяц")
    fig.update_yaxes(title_text=title)
    fig.update_layout(
       font_family="Calibri",
       font_color="black",
        font_size=14,
       title_font_family="Times New Roman",
       title_font_color="red",
       legend_title_font_color="green"
)

In [21]:
import plotly.graph_objects as go
fig = go.Figure()
for_city(fig,df_all,"среднее","математическое ожидание")
fig.show()

In [22]:
fig = go.Figure()
for_city(fig,df_all,"дисперсия","дисперсия")
fig.show()

In [91]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=2, cols=1)
for_city(fig,df_all,"среднее","математическое ожидание",1)
for_city(fig,df_all,"дисперсия","дисперсия",2)
fig.show(height=1000)